## Neural Network of 1 hidden layer using TensorFlow

#### Data preparations done in file LogReg-Sklearn.ipynb

In [1]:
# Importing libraries

from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
print("All libs imported.")

All libs imported.


In [2]:
# Loading the dataset

pickle_file = './data/notMNIST.pickle' # Change this to match the file location

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

FileNotFoundError: [Errno 2] No such file or directory: './data/notMNIST.pickle'

In [ ]:
# Reshaping the dataset as per requirements

image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

### Using 1 hidden layer neural net with ReLU 

In [ ]:
batch_size = 128
hidden_layer_size = 1024
learn_rate = 0.2

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size), name='Train_dataset_ph')
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels), name='Train_labels_ph')
  tf_valid_dataset = tf.constant(valid_dataset, name='Validation_dataset')
  tf_test_dataset = tf.constant(test_dataset, name='Test_dataset')
  
  # Variables.
  hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size,hidden_layer_size]))
  hidden_biases = tf.Variable(tf.zeros([hidden_layer_size]))

  output_weights = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation: Hidden Layer
  hidden_logits = tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases
  hidden_activation = tf.nn.relu(hidden_logits)

  # Output Layer
  output_logits = tf.matmul(hidden_activation, output_weights) + output_biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=output_logits))
    
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learn_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(output_logits)
  
  valid_activation = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_activation, output_weights) + output_biases)
    
  test_activation = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_activation, output_weights) + output_biases)

In [ ]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("\n\nStep %d" % step)
      print("Minibatch loss: %f" % (l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(session=session), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(session=session), test_labels))